<a href="https://colab.research.google.com/github/AryanSant27/Lora-Therapist/blob/main/Lora_YourOwnTherapy_Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers peft accelerate bitsandbytes


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
!unzip saved_model.zip -d adapters


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_model_name = "meta-llama/Llama-2-7b-hf"  # replace if different

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, "./adapters/saved_model")


In [ ]:
system_prompt = (
    "Your name is Lora"
    "You are a compassionate and professional therapist. "
    "You listen actively, validate emotions, and offer supportive, non-judgmental guidance. "
    "Respond in a calm, empathetic tone, focusing on helping the user reflect and grow emotionally. "
    "Always keep your responses appropriate, kind, and rooted in psychological best practices."
)

import torch

def generate_therapy_response(user_input: str):
    prompt = f"{system_prompt}\nUser: {user_input}\nTherapist:"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Strip off the prompt from the generated text
    response_only = response.split("Therapist:")[-1].strip()

    return response_only





In [ ]:
response = generate_therapy_response("I feel demotivated because of unemployment")
print(response)

In [ ]:
# If you want to clear the internal cache from previous inference (rarely needed)
torch.cuda.empty_cache()
